In [ ]:
!pip install flopco-pytorch

In [ ]:
import hashlib
import torch
from torch import nn
import flopco
from flopco import FlopCo

In [ ]:
name = input('Enter your full name and press Enter:')
randseed_str = hashlib.sha256(name.encode('utf-8')).hexdigest()
randseed = int(randseed_str[:10], 16)
print(randseed)

Enter your full name and press Enter:Chernyy Vladimir
505329706951


## Task 1 [1 point]

Convert bytes to KB and MB 

In [ ]:
n_bytes = randseed
n_KB = n_bytes / 2**10
n_MB = n_KB / 2**10

In [ ]:
n_KB

493486041.94433594

In [ ]:
n_MB

481919.96283626556

## Task 2 [1 point]

Let $X$ and $Y$ be an input and output vectors to fully-connected layer of the following type:
 $$Y = layer(X) = W^T@X,$$
 where $W$ is a weight matrix of size $N_{in} \times N_{out}$
 (in this task,  bias = None)

 Compute
 - FLOP
 - MAC

Let's replace $N_{in}$ and $N_{out}$ with just $n$ and $m$ correspondigly to increase readability.  

Given: $$W = \begin{pmatrix}
w^0_0 & w^1_0 & \cdots & w^{m -1}_0 \\
w^0_1 & w^1_1 & \cdots & w^{m -1}_1 \\
\vdots & \vdots & \vdots & \vdots \\
w^0_{n-1} & w^1_{n-1} & \cdots & w^{m-1}_{n-1}
\end{pmatrix}
, X = \begin{pmatrix}
x_0 \\
x_1 \\
\vdots \\
x_{n-1}
\end{pmatrix}
$$

Where lower indices in $W$ correspond to feature's id, upper represent neuron's id.

Thus, $$Y = \left\{ w^j \cdot X\right\}_{j=0}^{m-1}$$

We could fairly say, that computation of $y_i$ takes $n$ multiplications and $(n - 1)$ summations.

Thus, overall number of computations for $Y$ like following:

**multiplications:** $m \cdot n\quad$ *OR* $\quad$ $N_{out} \cdot N_{in}$

**summations:** $m \cdot (n - 1)\quad$ *OR* $\quad$ $N_{out} \cdot (N_{in} - 1)$

Let's translate in for conventional units:

**FLOPS** $= N_{out} \cdot N_{in} + N_{out} \cdot (N_{in} - 1) = N_{out} \cdot (2N_{in} - 1)$

**MAC** $= N_{out} \cdot N_{in}\quad$ same logic, but after we found scalar product we consider every pair of $(+, *)$ as "1" when accumulating$

# Task 3 [3 points]
Consider a convolutional layer, such that

- $H_{in} = W_{in} = 11$
- $k_{in} = k_{out} = 3$
- $C_{in} = C_{out} = 16$

Assume $H_{out} = W_{out} = 9$

a) [1 point] $padding_{h} = padding_{w}$ - ?

b) [1 point] FLOP and MAC for standard convolution - ? 

c) [1 point] FLOP and MAC for depth-wise convolution -?

**a)**

Using formula from [article](https://medium.com/analytics-vidhya/convolution-padding-stride-and-pooling-in-cnn-13dc1f3ada26) for our case:

$H_{out} = H_{in} + 2 \cdot \text{padding}_{h} - k_{in}+1$

$W_{out} = W_{in} + 2 \cdot \text{padding}_{w} - k_{in}+1$

Substituting known variables:

\begin{cases} 
11 + 2 \cdot \text{padding}_{h} - 3 + 1 = 9\\
11 + 2 \cdot \text{padding}_{w} - 3 + 1 = 9
\end{cases}

Expressing unknown:

\begin{cases} 
2 \cdot \text{padding}_{h} = 0\\
2 \cdot \text{padding}_{w} = 0
\end{cases}

It's only possible in case, when padding_{h} = padding_{w} = 0

Answer: **yes**

**c)**

Number of matrix product positions $=W_{out} \cdot H_{out}$

Every iteration of computing convolution matrix is just elementwise products and following summation:

Number of matrix product positions $=W_{out} \cdot H_{out}$

Every iteration of computing convolution matrix is just elementwise products and following summation.

**multiplications:** $C_{out} \cdot W_{out} \cdot H_{out}  \cdot k_{in} \cdot k_{out}$

**summations:** $C_{out} \cdot W_{out} \cdot H_{out}  \cdot (k_{in} \cdot k_{out} - 1)$ 

Answer:

Let's translate in for conventional units:

**FLOPS** $= C_{out} \cdot W_{out} \cdot H_{out}  \cdot(2 k_{in} \cdot k_{out} - 1)$

**MAC** $= C_{out} \cdot W_{out} \cdot H_{out}  \cdot k_{in} \cdot k_{out}$

- $H_{in} = W_{in} = 11$
- $k_{in} = k_{out} = 3$
- $C_{in} = C_{out} = 16$

Assume $H_{out} = W_{out} = 9$

$\Rightarrow$ **FLOPS** $= C_{out} \cdot W_{out} \cdot H_{out}  \cdot(2 k_{in} \cdot k_{out} - 1) = 16 \cdot 9^2(2 \cdot 3^2 - 1) = 22032$


$\Rightarrow$ **MAC** $= C_{out} \cdot W_{out} \cdot H_{out}  \cdot k_{in} \cdot k_{out} = 16 \cdot 9^2 \cdot 3^2 = 11664$

**b)**

The only diffence between **b)** and **c)** is that in **b)** for every output 2D layer we compute 2D matrix elemetwise sum $C_{in}$ times after kernelwise convolutions

**multiplications:** $C_{in} \cdot C_{out} \cdot W_{out} \cdot H_{out}  \cdot k_{in} \cdot k_{out}$

**summations:** $(C_{in} - 1) \cdot  C_{out} \cdot W_{out} \cdot H_{out}  \cdot (k_{in} \cdot k_{out} - 1)$ 

Answer:

Let's translate in for conventional units:

**FLOPS** $= C_{out} \cdot W_{out} \cdot H_{out}[C_{in} \cdot(2 k_{in} \cdot k_{out} - 1) - (k_{in} \cdot k_{out} - 1)]$ $= C_{out} \cdot W_{out} \cdot H_{out}[k_{in} \cdot k_{out}(2C_{in} - 1) - C_{in} + 1]$

**MAC** $= C_{in} \cdot C_{out} \cdot W_{out} \cdot H_{out}  \cdot k_{in} \cdot k_{out}$

- $H_{in} = W_{in} = 11$
- $k_{in} = k_{out} = 3$
- $C_{in} = C_{out} = 16$

Assume $H_{out} = W_{out} = 9$

$\Rightarrow$ **FLOPS** $= C_{out} \cdot W_{out} \cdot H_{out}[k_{in} \cdot k_{out}(2C_{in} - 1) - C_{in} + 1] = 16 \cdot 9^2[3^2(2 \cdot 16 - 1) - 16 + 1] = 342144$

$\Rightarrow$ **MAC** $= C_{in} \cdot C_{out} \cdot W_{out} \cdot H_{out}  \cdot k_{in} \cdot k_{out} =16^2 \cdot 9^2 \cdot 3^2 = 186624$

# Task 4 [2 points]
Assume that size of input image $x$ is $(3, 220, 220)$.

Compute FLOP for BatchNorm2d(x)

Note: Consider, that  FLOP(division operation) = 4.

Assume, that batch size = 10

Mean takes: $\underbrace{4}_{division} + \underbrace{220^2 - 1}_{2d \space matrix \space sum}$

Std takes: $\underbrace{4}_{\sqrt{}} + \underbrace{4}_{division} + \underbrace{2 \cdot 220^2}_{-\mu \space + squared} + \underbrace{220^2 - 1}_{2d \space matrix \space sum} + \underbrace{1}_{\varepsilon}$

In the end we subtract mean and divide by std and multiply and shift:$(\underbrace{1}_{-\mu} + \underbrace{4}_{division} + \underbrace{2}_{\gamma + \beta}) \cdot 220^2$

We computed batch-wise, now let's summarize:

For every channel we compute our statistics:

Channels num = $3$

**FLOP** $= 3  \cdot ((4 + 220^2 - 1) + (4 + 4 + 2 \cdot 220^2 + 220^2 - 1 + 1) + (1 + 4 + 2) \cdot 220^2)) = 5808000$

In [6]:
3 * ((3 + 220**2) + (8 + 2 * 220**2) + (7 * 220**2))

1452033

In [1]:
!pip install torch torchvision tensorly==0.4.5 flopco-pytorch

     |████████████████████████████████| 70 kB 5.3 MB/s 
     |████████████████████████████████| 154 kB 30.7 MB/s 
  Created wheel for tensorly: filename=tensorly-0.4.5-py3-none-any.whl size=100163 sha256=cfd1b45b16a5af990604eb6cacc12da5cc5db538adc46edf5dd467fcb01bed5c
  Stored in directory: /root/.cache/pip/wheels/05/ed/36/493bba3faa150a1193eec864db4951355eb64659330cb00722
Successfully built tensorly


In [35]:
x = torch.rand((1, 220, 220, 3))

In [49]:
layer = nn.BatchNorm2d(num_features=220)

In [58]:
ord_stats = FlopCo(layer, x.shape, instances=[nn.BatchNorm2d])
ord_stats.total_flops

580800

# Task 5 [3 points]
 a) [1 point] Compute FLOP for Parametric ReLU (PReLU):

$$\operatorname{PReLU}
\left(y_{i}\right)= \begin{cases}y_{i}, & \text { if } y_{i}>0 \\ a_{i} y_{i}, & \text { if } y_{i} \leq 0\end{cases}$$

In [ ]:
### YOUR FORMULA HERE

 b) [1 point] Compute FLOP for Gaussian Error Linear Units function (GELU):

$$\operatorname{GELU}(x)=x * \Phi(x) = 0.5 x\left(1+\tanh \left(\sqrt{2 / \pi}\left(x+0.044715 x^{3}\right)\right)\right),$$

where $\Phi(x)$ - is the Cumulative Distribution Function for Gaussian Distribution.

Note: consider, that $\operatorname{FLOP}(\operatorname{tanh}) = 60$.

In [ ]:
### YOUR FORMULA HERE

c) [1 point] Compute FLOP for Softmax function:


$$\operatorname{Softmax}\left(x_{i}\right)=\frac{\exp \left(x_{i}\right)}{\sum_{j} \exp \left(x_{j}\right)}.$$

Note: consider, that $\operatorname{FLOP}(\operatorname{exp}) = 40$.

In [ ]:
### YOUR FORMULA HERE